In [13]:
#importing libraries
from ast import Global
from textwrap import wrap
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tkinter import *
from tkinter import ttk


#reading dataset
videos_df = pd.read_csv('youtube.csv')

#data pre-processing
selected_features = ['link', 'title', 'channel', 'category']
for feature in selected_features:
    videos_df[feature] = videos_df[feature].fillna('')
combined_features = videos_df['link'] + " " + videos_df['title'] + " " + videos_df['channel'] + " " + videos_df['category']
#print(combined_features)

#converting textual data to numeric features
Vectorizer = TfidfVectorizer()
features = Vectorizer.fit_transform(combined_features)
#print(features)

#cosine similarity
similarity = cosine_similarity(features)
#print(similarity)

#recommender function
def recommender(video_name):
    recommended = []
    list_of_videos = videos_df['title'].tolist()
    close_matches = difflib.get_close_matches(video_name, list_of_videos, cutoff = 0.4)
    video = close_matches[0]
    index = videos_df[videos_df['title'] == video]['index'].values[0]
    similarity_score = list(enumerate(similarity[index]))
    sorted_similar_videos = sorted(similarity_score, key = lambda x:x[1], reverse = True)
    i=1
    for video in sorted_similar_videos:
        #video_title = video_df[video_df.index == video[0]]['title'].values[0]
        if (i<12):
            recommended.append(video[0])
            i+=1
    return recommended[1:]
       
# defining main window
root = Tk()
root.title("YouTube Video Recommender")
root.state("zoomed")
root.configure(bg = "black")

#find movies function
recommendations = []
def find():
    global choice
    global recommendations
    global L
    
    c = choice.get()
    choice.delete(0, END)
    recommendations = recommender(c)
    print(recommendations)

    #defining new window
    new_window = Toplevel(root)
    new_window.title("Recommendations")
    new_window.state("zoomed")

    new_window.configure(bg = "black")

    frame1 = LabelFrame(new_window, padx = 20, pady = 20, bg = "red")
    frame1.pack(padx = 20, pady = 20)

    #defining prev next functions
    def prev_video(videonumber):
        global L

        L.grid_forget()

        status_bar = Label(frame1, text = "Video " + str(videonumber) + " of " + '10', bd = 2, relief = SUNKEN, padx = 5, pady = 5, bg = "black", fg = "white")
        status_bar.configure(font=("bold", 20))
        status_bar.grid(row = 0, column = 1, sticky = W + E, padx = 10, pady = 10)  

        prev_ = Button(frame1, text = "<<",bg = "white", fg = "black", borderwidth = 5, padx = 20, pady = 20, command=lambda : prev_video(videonumber-1))
        prev_.configure(font = (20))

        next_ = Button(frame1, text = ">>",bg = "white", fg = "black", borderwidth = 5, padx = 20, pady = 20, command = lambda : next_video(videonumber+1))
        next_.configure(font = (20))

        prev_.grid(row = 1, column = 0, padx = 20, pady = 20)
        next_.grid(row = 1, column = 2, padx = 20, pady = 20) 

        i = videonumber-1
        title =  videos_df[videos_df.index == recommendations[i]]['title'].values[0]
        channel = videos_df[videos_df.index == recommendations[i]]['channel'].values[0][:300]
        genres = videos_df[videos_df.index == recommendations[i]]['category'].values[0]

        video_data = "Title : " + title + '\n' + "Genres : " + genres + "\n\n" + "-----CHANNEL-----" + "\n\n" + channel + "...."
        L = Label(frame1, text = video_data, wraplength= 500, padx = 20, pady = 20, bg = "red", fg = "white", font = ("Times", 20, "bold"))
        L.grid(row = 1, column = 1)


        if(videonumber == 1):
            prev_ = Button(frame1, text = "<<",bg = "white", fg = "black", borderwidth = 5, padx = 20, pady = 20, command=lambda : prev_video(videonumber-1), state=DISABLED)
            prev_.configure(font = (20))
            prev_.grid(row = 1, column = 0, padx = 20, pady = 20)

    
    def next_video(videonumber):
        global L

        L.grid_forget()

        status_bar = Label(frame1, text = "Video " + str(videonumber) + " of " + '10', bd = 2, relief = SUNKEN, padx = 5, pady = 5, bg = "black", fg = "white")
        status_bar.configure(font=("bold", 20))
        status_bar.grid(row = 0, column = 1, sticky = W + E, padx = 10, pady = 10)  

        prev_ = Button(frame1, text = "<<",bg = "white", fg = "black", borderwidth = 5, padx = 20, pady = 20, command=lambda : prev_video(videonumber-1))
        prev_.configure(font = (20))

        next_ = Button(frame1, text = ">>",bg = "white", fg = "black", borderwidth = 5, padx = 20, pady = 20, command = lambda : next_video(videonumber+1))
        next_.configure(font = (20))

        prev_.grid(row = 1, column = 0, padx = 20, pady = 20)
        next_.grid(row = 1, column = 2, padx = 20, pady = 20) 

        i = videonumber-1
        title =  videos_df[videos_df.index == recommendations[i]]['title'].values[0]
        channel = videos_df[videos_df.index == recommendations[i]]['channel'].values[0][:300]
        genres = videos_df[videos_df.index == recommendations[i]]['category'].values[0]

        video_data = "Title : " + title + '\n' + "Genres : " + genres + "\n\n" + "-----CHANNEL-----" + "\n\n" + channel + "...."
        L = Label(frame1, text = video_data, wraplength= 500, padx = 20, pady = 20, bg = "red", fg = "white", font = ("Times", 20, "bold"))
        L.grid(row = 1, column = 1)

        if(videonumber==10):
            next_ = Button(frame1, text = ">>",bg = "white", fg = "black", borderwidth = 5, padx = 20, pady = 20, command = lambda : next_video(videonumber+1), state=DISABLED)
            next_.configure(font = (20))
            next_.grid(row = 1, column = 2, padx = 20, pady = 20) 


    # defining display of second window
    status_bar = Label(frame1, text = "Video 1 of " + '10', bd = 2, relief = SUNKEN, padx = 5, pady = 5, bg = "black", fg = "white")
    status_bar.configure(font=("bold", 20))
    status_bar.grid(row = 0, column = 1, sticky = W + E, padx = 10, pady = 10)  
    
    prev_ = Button(frame1, text = "<<",bg = "white", fg = "black", borderwidth = 5, padx = 20, pady = 20, command=lambda : prev_video(), state = DISABLED)
    prev_.configure(font = (20))
    
    next_ = Button(frame1, text = ">>",bg = "white", fg = "black", borderwidth = 5, padx = 20, pady = 20, command = lambda : next_video(2))
    next_.configure(font = (20))
    
    prev_.grid(row = 1, column = 0, padx = 20, pady = 20)
    next_.grid(row = 1, column = 2, padx = 20, pady = 20)  

    i = 0
    title =  videos_df[videos_df.index == recommendations[i]]['title'].values[0]
    channel = videos_df[videos_df.index == recommendations[i]]['channel'].values[0][:300]
    genres = videos_df[videos_df.index == recommendations[i]]['category'].values[0]

    video_data = "Title : " + title + '\n' + "Genres : " + genres + "\n\n" + "-----CHANNEL-----" + "\n\n" + channel + "...."
    
    L = Label(frame1, text = video_data, wraplength= 500, padx = 20, pady = 20, bg = "red", fg = "white", font = ("Times", 20, "bold"))
    L.grid(row = 1, column = 1)

#button functions
def on_enter(e):
   button.config(background='black', foreground= "white")

def on_leave(e):
   button.config(background= '#454545', foreground= 'white')

#defining frame
frame = LabelFrame(root, padx = 20, pady = 20, bg = "red")
frame.pack(padx = 20, pady = 20)

#defining app header
heading = Label(frame, text = "YOUTUBE VIDEO RECOMMENDER", bg = "red")
heading.configure(font = ('Times',60, "bold"))
heading.grid(row = 0, column = 0, columnspan = 3, padx = 40, pady = 40)

Label(frame, text = "ENTER YOUR LAST WATCHED VIDEO NAME", padx = 20, pady = 20, bg = "red", fg = "white", font = ("Times", 30, "bold")).grid(row = 1, column = 1, padx = 20, pady = 30)

choice = Entry(frame, width = 50, borderwidth = 5)
choice.configure(font = ("Times", 30, "bold"))
choice.grid(row = 2, column = 1, padx = 10, pady = 10, ipadx = 30, ipady = 10)

button = Button(frame, text = "FIND NEXT VIDEOS", padx = 10, pady = 10, borderwidth = 10, relief = RAISED,bg="#454545", fg="white", command = find)
button.configure(font = ("Times", 20, "bold"))
button.grid(row = 3, column = 1, padx = 10, pady = (50, 10), ipadx = 40, ipady = 5)

button.bind('<Enter>', on_enter)
button.bind('<Leave>', on_leave)

# mainloop which run infinitely
root.mainloop()

[3116, 3533, 3428, 3423, 3548, 3485, 3560, 3454, 3406, 3529]
